In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils

from torch.autograd import Variable
from torchvision import datasets, models, transforms

In [2]:
import numpy as np
import os

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [5]:
data_dir = 'data'

In [6]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}

In [7]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=24, shuffle=True) for x in ['train', 'val']}

In [29]:
dataloaders['train']

In [8]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
dataset_sizes

{'train': 4279, 'val': 471}

In [9]:
class_names = image_datasets['train'].classes
class_names

['Black-grass',
 'Charlock',
 'Cleavers',
 'Common Chickweed',
 'Common wheat',
 'Fat Hen',
 'Loose Silky-bent',
 'Maize',
 'Scentless Mayweed',
 'Shepherds Purse',
 'Small-flowered Cranesbill',
 'Sugar beet']

In [10]:
model = models.vgg16(pretrained=True)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, len(class_names))

model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [11]:
criterion = nn.CrossEntropyLoss()

In [12]:
optimizer = optim.Adam(model.parameters(), lr=0.003)

In [13]:
exp_lr_decay = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

In [17]:
def train(model, dataloader, optimizer, decay, criterion, epoch):
    model.train()
    
    for batch_idx, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)
        
        print(y)

        optimizer.zero_grad()
        prediction = model(x)

        loss = criterion(prediction, y)
        loss.backward()

        optimizer.step() 
        
        if batch_idx % 1000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(x), len(dataloader.dataset),
                100. * batch_idx / len(dataloader), loss.data.item()))

In [18]:
def val(model, dataloader, criterion):
    model.eval()
    
    val_loss = 0
    correct = 0
    
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            
            prediction = model(x)
            val_loss += criterion(prediction, y, size_average=False).item()
            
            y_hat = prediction.max(1, keepdim=True)[1]
            correct += y_hat.eq(y.view_as(y_hat)).sum().item()
    
    val_loss /= len(dataloader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(dataloader.dataset),
        100. * correct / len(dataloader.dataset)))

In [19]:
for epoch in range(1, 5+1):
    train(model, dataloaders['train'], optimizer, exp_lr_decay, criterion, epoch)
    val(model, dataloaders['val'], criterion)

tensor([  1,   5,   1,  11,   1,  10,   7,   5,   5,   7,   8,   3,
         11,  10,   8,   1,   5,   8,  11,   2,   3,  10,   6,   9], device='cuda:0')
Train Epoch: 1 [0/4279 (0%)]	Loss: 2.775584
tensor([  1,   3,   8,   5,   9,   1,   2,  10,   6,   2,   3,   9,
          5,   4,   5,   5,   0,   8,   6,   1,  10,   1,   3,   8], device='cuda:0')
tensor([  7,   6,   5,   4,   6,   8,   6,   1,   8,   5,   2,   1,
          6,   4,   1,   7,  10,   4,   8,   1,   9,   8,   1,   6], device='cuda:0')
tensor([  5,   8,   1,   6,   8,   1,  10,  10,   2,   8,   9,  11,
         11,   8,   0,   8,   5,   3,   5,   9,   3,   8,   2,   6], device='cuda:0')
tensor([  6,   8,   0,   8,   3,   1,  10,   4,   5,   0,   2,   4,
          8,  11,   0,   8,   1,   8,   2,   8,   8,  11,   4,   5], device='cuda:0')
tensor([  1,   5,   0,  11,   3,   5,   3,   3,   2,  10,  11,   8,
          4,   7,  10,   5,   0,  10,   7,   8,  11,   0,   4,   3], device='cuda:0')
tensor([  9,   3,   4,   3,   3,

KeyboardInterrupt: 